
## Query 2

### Find data from all patients who have been treated with "Radiation Therapy, NOS" and have both genomic and proteomic data

In this notebook, the question "and have both genomic and and proteomic data" is asked as "and have data in both the genomic and proteomic data commons". That is a useful thing to do in testing CDA as it represents a true aggregation use case across CRDC nodes. However, we should not lose sight of the fact that the query examples here are not expressed in terms of the type of data being queried. Cases already exist in CRDC where data of a given type are present in more than one node See [related workbook](./Query2%20by%20datatype.ipynb) for an exploration of that. That point made, on with the examples that query according to the node in which data resides.

The queries here start with the examples posted in this workbook https://github.com/pihltd/CDA-Testing/blob/master/OfficialCDATesting.ipynb

Set up for SQL Queries using cda_client.

In [1]:
import cda_client
host='https://cda.cda-dev.broadinstitute.org'
api_client = cda_client.ApiClient(configuration=cda_client.Configuration(host=host))
api_instance = cda_client.QueryApi(api_client)

Using the exploratory work in the source notebook we can jumop straight to the SQL examples used there. It seems direct SQL was necessary to fulfill this query.

The following query is the same as in the source notebook and gives the same results.

In [5]:
query1 = (
    '''SELECT COUNT(DISTINCT _ResearchSubject.id) as rsidcount 
     FROM gdc-bq-sample.cda_mvp.v3,
     UNNEST(ResearchSubject) AS _ResearchSubject, 
     UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis, 
     UNNEST(_Diagnosis.Treatment) AS _Treatment 
     WHERE _Treatment.type = "Radiation Therapy, NOS" '''
)
api_instance.sql_query('v3',query1)

{'next_url': None,
 'previous_url': None,
 'query_sql': 'SELECT COUNT(DISTINCT _ResearchSubject.id) as rsidcount FROM '
              'gdc-bq-sample.cda_mvp.v3,\n'
              '     UNNEST(ResearchSubject) AS _ResearchSubject, '
              'UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis, '
              'UNNEST(_Diagnosis.Treatment) AS _Treatment \n'
              '     WHERE _Treatment.type = "Radiation Therapy, NOS" ',
 'result': [{'rsidcount': 11507}]}

Our first modification is to count patients we should query for distinct patient ids rather than subject ids.

In [8]:
query2 = (
    '''SELECT COUNT(DISTINCT p.id) as rsidcount FROM gdc-bq-sample.cda_mvp.v3 p,
     UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis, UNNEST(_Diagnosis.Treatment) AS _Treatment 
     WHERE _Treatment.type = "Radiation Therapy, NOS" '''
)
api_instance.sql_query('v3',query2)

{'next_url': None,
 'previous_url': None,
 'query_sql': 'SELECT COUNT(DISTINCT p.id) as rsidcount FROM '
              'gdc-bq-sample.cda_mvp.v3 p,\n'
              '     UNNEST(ResearchSubject) AS _ResearchSubject, '
              'UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis, '
              'UNNEST(_Diagnosis.Treatment) AS _Treatment \n'
              '     WHERE _Treatment.type = "Radiation Therapy, NOS" ',
 'result': [{'rsidcount': 11507}]}

Again skipping some exploratory steps in the source workbook, we can jump straight to the combination query which seems to give a sensible result to the intended use  case.

In [14]:
comboquery = '''SELECT COUNT(DISTINCT _ResearchSubject.id) 
FROM (SELECT * FROM gdc-bq-sample.cda_mvp.v3, 
    UNNEST(ResearchSubject) AS _ResearchSubject, 
    UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis, 
    UNNEST(_Diagnosis.Treatment) AS _Treatment, 
    UNNEST(_ResearchSubject.Specimen) AS _Specimen, 
    UNNEST(_Specimen.identifier) as _identifier 
    WHERE ((_Treatment.type = "Radiation Therapy, NOS") AND (_identifier.system = "GDC"))
    ),
    UNNEST(ResearchSubject) AS _ResearchSubject, 
    UNNEST(_ResearchSubject.identifier) AS _identifier 
    WHERE (_identifier.system = "PDC")'''
api_instance.sql_query('v3',comboquery)

{'next_url': None,
 'previous_url': None,
 'query_sql': 'SELECT COUNT(DISTINCT _ResearchSubject.id) FROM (SELECT * FROM '
              'gdc-bq-sample.cda_mvp.v3, \n'
              '    UNNEST(ResearchSubject) AS _ResearchSubject, '
              'UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis, '
              'UNNEST(_Diagnosis.Treatment) AS _Treatment, \n'
              '    UNNEST(_ResearchSubject.Specimen) AS _Specimen, '
              'UNNEST(_Specimen.identifier) as _identifier \n'
              '    WHERE ((_Treatment.type = "Radiation Therapy, NOS") AND '
              '(_identifier.system = "GDC"))), \n'
              '    UNNEST(ResearchSubject) AS _ResearchSubject, '
              'UNNEST(_ResearchSubject.identifier) AS _identifier WHERE '
              '(_identifier.system = "PDC")',
 'result': [{'': 369}]}

Again modifying the query to count patients rather than subjects.

In [17]:
comboquery2 = '''SELECT COUNT(DISTINCT pa.id) FROM 
                    (SELECT p.* FROM gdc-bq-sample.cda_mvp.v3 p, 
                        UNNEST(ResearchSubject) AS _ResearchSubject, 
                        UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis, 
                        UNNEST(_Diagnosis.Treatment) AS _Treatment, 
                        UNNEST(_ResearchSubject.Specimen) AS _Specimen, 
                        UNNEST(_Specimen.identifier) as _identifier 
                        WHERE ((_Treatment.type = "Radiation Therapy, NOS") AND (_identifier.system = "GDC"))
                    ) pa, 
                    UNNEST(ResearchSubject) AS _ResearchSubject, 
                    UNNEST(_ResearchSubject.identifier) AS _identifier WHERE 
                    (_identifier.system = "PDC")'''
api_instance.sql_query('v3',comboquery2)

{'next_url': None,
 'previous_url': None,
 'query_sql': 'SELECT COUNT(DISTINCT pp.id) FROM \n'
              '                    (SELECT p.* FROM gdc-bq-sample.cda_mvp.v3 '
              'p, \n'
              '                        UNNEST(ResearchSubject) AS '
              '_ResearchSubject, \n'
              '                        UNNEST(_ResearchSubject.Diagnosis) AS '
              '_Diagnosis, \n'
              '                        UNNEST(_Diagnosis.Treatment) AS '
              '_Treatment, \n'
              '                        UNNEST(_ResearchSubject.Specimen) AS '
              '_Specimen, \n'
              '                        UNNEST(_Specimen.identifier) as '
              '_identifier \n'
              '                        WHERE ((_Treatment.type = "Radiation '
              'Therapy, NOS") AND (_identifier.system = "GDC"))\n'
              '                    ) pp, \n'
              '                    UNNEST(ResearchSubject) AS '
              '_Res

There is a discrepanvy in how the query is done for GDC and PDC. The GDC identifier query is done on the Specimen identifier while the PDC query uses the subject identifier.
We can remove an unnest from the query by using the ResearchSubject identifier. 

* there are issues with this. In fact in TCGA/CPTC it is the specimens that were specific to Genomics and Proteomics. Physically distinct materials (whether we choose to call them specimens, portions or aliquots) are what go to for analysis on different platforms and results in different types of data.

However, it was not straightforward to create this query. Refer to the source notebook for the  preliminary queries to used to determine that the treatment type was only specified for GDC Research Subjects. 

This is important to formulating the correct query. The following is the same query but with PDC and GDC swapped. No matches occur. 

In [39]:
comboquery2b =  '''SELECT COUNT(DISTINCT pa.id) FROM 
                    (SELECT p.* FROM gdc-bq-sample.cda_mvp.v3 p, 
                        UNNEST(ResearchSubject) AS _ResearchSubject, 
                        UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis, 
                        UNNEST(_Diagnosis.Treatment) AS _Treatment, 
                        UNNEST(_ResearchSubject.identifier) AS _identifier 
                        WHERE ((_Treatment.type = "Radiation Therapy, NOS") AND (_identifier.system = "PDC"))
                    ) pa, 
                    UNNEST(ResearchSubject) AS _ResearchSubject, 
                    UNNEST(_ResearchSubject.identifier) AS _identifier WHERE 
                    (_identifier.system = "GDC")'''
api_instance.sql_query('v3',comboquery2b)

{'next_url': None,
 'previous_url': None,
 'query_sql': 'SELECT COUNT(DISTINCT pa.id) FROM \n'
              '                    (SELECT p.* FROM gdc-bq-sample.cda_mvp.v3 '
              'p, \n'
              '                        UNNEST(ResearchSubject) AS '
              '_ResearchSubject, \n'
              '                        UNNEST(_ResearchSubject.Diagnosis) AS '
              '_Diagnosis, \n'
              '                        UNNEST(_Diagnosis.Treatment) AS '
              '_Treatment, \n'
              '                        UNNEST(_ResearchSubject.Specimen) AS '
              '_Specimen, \n'
              '                        UNNEST(_Specimen.identifier) as '
              '_identifier \n'
              '                        WHERE ((_Treatment.type = "Radiation '
              'Therapy, NOS") AND (_identifier.system = "PDC"))\n'
              '                    ) pa, \n'
              '                    UNNEST(ResearchSubject) AS '
              '_Res

We imagine that the following might fix the problem. This should work whichever of the ResearchSubjects has the Diagnosis and Treatment.

In [69]:
comboquery2c =  '''SELECT COUNT(DISTINCT p.id) patientCount FROM 
                    gdc-bq-sample.cda_mvp.v3 p,
                    
                    UNNEST(p.ResearchSubject) AS _ResearchSubject, 
                    UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis, 
                    UNNEST(_Diagnosis.Treatment) AS _Treatment, 
                    UNNEST(_ResearchSubject.identifier) AS _identifier ,
                    
                    UNNEST(p.ResearchSubject) AS _ResearchSubject2, 
                    UNNEST(_ResearchSubject2.Diagnosis) AS _Diagnosis2, 
                    UNNEST(_Diagnosis2.Treatment) AS _Treatment2,
                    UNNEST(_ResearchSubject2.identifier) AS _identifier2
                    
                    WHERE (
                    ( (_identifier.system = "GDC") AND (_identifier2.system = "PDC") )
                    AND
                    ( (_Treatment.type = "Radiation Therapy, NOS") OR (_Treatment2.type = "Radiation Therapy, NOS"))
                    )'''

api_instance.sql_query('v3',comboquery2c)

{'next_url': None,
 'previous_url': None,
 'query_sql': 'SELECT COUNT(DISTINCT p.id) patientCount FROM \n'
              '                    gdc-bq-sample.cda_mvp.v3 p, \n'
              '                        UNNEST(p.ResearchSubject) AS '
              '_ResearchSubject, \n'
              '                        UNNEST(_ResearchSubject.Diagnosis) AS '
              '_Diagnosis, \n'
              '                        UNNEST(_Diagnosis.Treatment) AS '
              '_Treatment, \n'
              '                        UNNEST(_ResearchSubject.identifier) AS '
              '_identifier ,\n'
              '                    UNNEST(p.ResearchSubject) AS '
              '_ResearchSubject2, \n'
              '                    UNNEST(_ResearchSubject2.Diagnosis) AS '
              '_Diagnosis2, \n'
              '                    UNNEST(_Diagnosis2.Treatment) AS '
              '_Treatment2,\n'
              '                    UNNEST(_ResearchSubject2.identifier) AS '
   

The issue is likely that there is no nested Diagnosis and/or Treatment within the PDC ResearchSubject - so the unnest does not generate any rows to match with the GDC subject.

We could continue to try and find workarounds for this, but it is probably better to take a step back.

All this is trying to work around a model that does not reflect what actually happened in these studies. There were not separete diagnoses performed for the data in the GDC and PDC data. The tree shown for 09CO022 is more of the reality. If the data model reflected this reality, and the treatment is at the Patient level rather than ResearchSubject the whole query is simplified. That is true both from a query language point of view and from the scientists mental msp of the studies.


For demo purposes we can create treatment at the patient level as follows. 

`select p.*,  _d1.Treatment as treatment
FROM gdc-bq-sample.cda_mvp.v3 p,
unnest(ResearchSubject) as _s1,
UNNEST(_s1.identifier) as _i1,
UNNEST(_s1.Diagnosis) as _d1
where _i1.system = 'GDC' `

We can create that table on the fly via a with clause and execute a relatively clean query against it. We'll hide our modified table in a function. 


In [2]:
def queryModV3(api_instance, query, limit=None):
    modQuery = '''
    with new_v3 as (select p.*,  d1.Treatment as treatment
    FROM gdc-bq-sample.cda_mvp.v3 p,
    unnest(ResearchSubject) as s1,
    UNNEST(s1.identifier) as i1,
    UNNEST(s1.Diagnosis) as d1
    where i1.system = 'GDC') 
    {}
    '''.format(query)
    return api_instance.sql_query('v3',modQuery, limit=10)

Now we can pretend we're querying against a better model of TCGA/CPTAC.

In [3]:
tq2 = '''
select count(distinct new_v3.id) patientCount
from new_v3, unnest(new_v3.treatment) as tx,

unnest(ResearchSubject) as s1,
UNNEST(s1.identifier) as i1,

unnest(ResearchSubject) as s2,
UNNEST(s2.identifier) as i2

where tx.type = 'Radiation Therapy, NOS'
and i1.system = 'GDC'
and i2.system = 'PDC'

'''
print(queryModV3(api_instance, tq2))

{'next_url': None,
 'previous_url': None,
 'query_sql': '\n'
              '    with new_v3 as (select p.*,  d1.Treatment as treatment\n'
              '    FROM gdc-bq-sample.cda_mvp.v3 p,\n'
              '    unnest(ResearchSubject) as s1,\n'
              '    UNNEST(s1.identifier) as i1,\n'
              '    UNNEST(s1.Diagnosis) as d1\n'
              "    where i1.system = 'GDC') \n"
              '    \n'
              'select count(distinct new_v3.id) patientCount\n'
              'from new_v3, unnest(new_v3.treatment) as tx,\n'
              '\n'
              'unnest(ResearchSubject) as s1,\n'
              'UNNEST(s1.identifier) as i1,\n'
              '\n'
              'unnest(ResearchSubject) as s2,\n'
              'UNNEST(s2.identifier) as i2\n'
              '\n'
              "where tx.type = 'Radiation Therapy, NOS'\n"
              "and i1.system = 'GDC'\n"
              "and i2.system = 'PDC'\n"
              '\n'
              '\n'
              '    ',
 'result'

We no longer have to pay careful attention to whether i1 or i2 has to be GDC and which is PDC, We get the same result either way.

In [ ]:
tq3 = '''
select count(distinct new_v3.id) patientCount
from new_v3, unnest(new_v3.treatment) as tx,

unnest(ResearchSubject) as s1,
UNNEST(s1.identifier) as i1,

unnest(ResearchSubject) as s2,
UNNEST(s2.identifier) as i2

where tx.type = 'Radiation Therapy, NOS'
and i1.system = 'PDC'
and i2.system = 'GDC'

'''
print(queryModV3(api_instance, tq3))

Note: putting the treatment at the patient level works for TCGA/CPTAC because there was only one diagnostic event. In many studies that won't be true. The required model will be slightly more complex than we were able to get away with here, but the required model is fairly well known (e.g. in MAGE, FuGE, HCA data model and OpenSpecimen) .  In some of these models the event has sometimes been called SpecimenCollectionEvent - representing some event on the study or treatment protocol. 

The 'with' statement creates that table for us on the fly. In reality the preferable solution would be that the CDA ETL creates it as part of a revised model.

Even better would be that in forward looking planned studies data managament would not create the problem in the first place.

### What are the studies which have both GDC and PDC data.

In [153]:
projectQuery = '''
select  s1.associated_project pdcproject, s2.associated_project gdcproject, count(*) common_subjects
from gdc-bq-sample.cda_mvp.v3,

unnest(ResearchSubject) as s1,
UNNEST(s1.identifier) as i1,

unnest(ResearchSubject) as s2,
UNNEST(s2.identifier) as i2

where i1.system = 'PDC'
and i2.system = 'GDC'

group by s1.associated_project, s2.associated_project

'''
gdc_pdc_projects = api_instance.sql_query('v3',projectQuery)
import pandas as pd
pd.DataFrame(gdc_pdc_projects.result)


,pdcproject,gdcproject,common_subjects
0,CPTAC3-Discovery,CPTAC-3,794
1,CPTAC-2,CPTAC-2,323
2,CPTAC-TCGA,TCGA-COAD,60
3,CPTAC-TCGA,TCGA-READ,30
4,CPTAC-TCGA,TCGA-OV,174
5,CPTAC-TCGA,TCGA-BRCA,105
6,Integrated Proteogenomic Characterization of H...,ORGANOID-PANCREATIC,70
7,Human Early-Onset Gastric Cancer - Korea Unive...,ORGANOID-PANCREATIC,70
